In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

# Table of Contents
0. [Read in Data](#read)
1. [Preprocessing](#preproc)   
    1.1 [Duplication Check](#duplic)  
    1.2 [NaN Values](#NaN)  
    1.3 [Creating the Type Chart](#chart)  
    1.4 [Creating the Type Multiplier Function](#multi)  
2. [Creating the Final Dataframe](#finalframe)
3. [Creating the Model / Prediction System](#predict)
4. [TRY IT OUT!](#testit)
5. [Future Steps](#future)

## Read in Data
<a id="read"></a>
___

In [2]:
# read in the csv's into a dataframe
pokemon = pd.read_csv('pokemon.csv')
combat = pd.read_csv('combats.csv')

In [3]:
pokemon.head()

#           Name Type 1  Type 2  HP  Attack  Defense  Sp. Atk  Sp. Def  \
0  1      Bulbasaur  Grass  Poison  45      49       49       65       65   
1  2        Ivysaur  Grass  Poison  60      62       63       80       80   
2  3       Venusaur  Grass  Poison  80      82       83      100      100   
3  4  Mega Venusaur  Grass  Poison  80     100      123      122      120   
4  5     Charmander   Fire     NaN  39      52       43       60       50   

   Speed  Generation  Legendary  
0     45           1      False  
1     60           1      False  
2     80           1      False  
3     80           1      False  
4     65           1      False

In [4]:
combat.head()

First_pokemon  Second_pokemon  Winner
0            266             298     298
1            702             701     701
2            191             668     668
3            237             683     683
4            151             231     151

For simplicity, let's turn these IDs into names.

In [5]:
# Zip the ID and Name into a new dict
names_dict = dict(zip(pokemon['#'], pokemon['Name']))

# Define variable 'columns'
cols = ["First_pokemon","Second_pokemon","Winner"]

# Create a new table using the columns of the previous combat table, replacing
# the ID with the name of the Pokemon from the dict
combat_name = combat[cols].replace(names_dict)
combat_name.head(3)

First_pokemon Second_pokemon     Winner
0      Larvitar        Nuzleaf    Nuzleaf
1      Virizion      Terrakion  Terrakion
2       Togetic       Beheeyem   Beheeyem

## Preprocessing
<a id="preproc"></a>
___

We should check the shape of the pokemon dataframes to get an idea of the size.

In [6]:
pokemon.shape

(800, 12)

In [7]:
combat_name.shape

(50000, 3)

#### Duplication Check
<a id="duplic"></a>
___

In [8]:
# Check for duplicate rows
pokemon.duplicated().sum()

0

No duplicates, so we can move on.

#### Checking for NaN values
<a id="NaN"></a>
___

In [9]:
# Check for NaN values
pokemon.isna().sum()

#               0
Name            1
Type 1          0
Type 2        386
HP              0
Attack          0
Defense         0
Sp. Atk         0
Sp. Def         0
Speed           0
Generation      0
Legendary       0
dtype: int64

There is one NaN value in the Pokemon `Name` column. Let's find it.

In [10]:
# Find rows with NaN values in Name
pokemon[pokemon['Name'].isna()]

# Name    Type 1 Type 2  HP  Attack  Defense  Sp. Atk  Sp. Def  Speed  \
62  63  NaN  Fighting    NaN  65     105       60       60       70     95   

    Generation  Legendary  
62           1      False

Pokemon number 63 is Primeape, so let's go ahead and input that.

In [11]:
# Locate row 62, and replace 'Name' column
pokemon.loc[62, "Name"] = "Primeape"

# Check
pokemon[pokemon['Name'].isna()]

Empty DataFrame
Columns: [#, Name, Type 1, Type 2, HP, Attack, Defense, Sp. Atk, Sp. Def, Speed, Generation, Legendary]
Index: []

There are still 386 NaN values in the `Type 2` column, so let's change these to 'None'.

In [12]:
# Fill any NaN values in 'Type 2' column with 'None'
pokemon['Type 2'].fillna('None', inplace=True)

# Check
pokemon.head(10)

#              Name Type 1  Type 2  HP  Attack  Defense  Sp. Atk  Sp. Def  \
0   1         Bulbasaur  Grass  Poison  45      49       49       65       65   
1   2           Ivysaur  Grass  Poison  60      62       63       80       80   
2   3          Venusaur  Grass  Poison  80      82       83      100      100   
3   4     Mega Venusaur  Grass  Poison  80     100      123      122      120   
4   5        Charmander   Fire    None  39      52       43       60       50   
5   6        Charmeleon   Fire    None  58      64       58       80       65   
6   7         Charizard   Fire  Flying  78      84       78      109       85   
7   8  Mega Charizard X   Fire  Dragon  78     130      111      130       85   
8   9  Mega Charizard Y   Fire  Flying  78     104       78      159      115   
9  10          Squirtle  Water    None  44      48       65       50       64   

   Speed  Generation  Legendary  
0     45           1      False  
1     60           1      False  
2     80           1      False  
3     80           1      False  
4     65           1      False  
5     80           1      False  
6    100           1      False  
7    100           1      False  
8    100           1      False  
9     43           1      False

#### Creating the Type Chart
<a id="chart"></a>
___

In Pokemon, there is a rather unique system involved that gives certain Pokemon distinct advantages over others in combat. This is called the Type system. Put simply, damage dealt from one type to another type could have a bonus multiplier based on the type advantage. For example, Fire is weak to Water, so a Water attack would have a 2x multiplier. While we are not looking at specific moves, it would be ideal to look at how certain types interact with one another. We will do this by creating a Type Chart.

In [13]:
# Create dicts for each variable, using TypeName: Multiplier. Multiplier
# assumes that the variable name is the attacker. (ex. Normal is the attacker
# in the first variable)
Normal = {"Normal": 1, "Fighting": 1, "Poison": 1, "Ground": 1, "Flying": 1, "Bug": 1, "Rock": 0.5, "Ghost": 0, "Steel": 0.5, "Fire": 1, "Water": 1, "Electric": 1, "Grass": 1, "Ice": 1, "Psychic": 1, "Dragon": 1, "Dark": 1, "Fairy": 1}
Fighting = {"Normal": 2, "Fighting": 1, "Poison": 0.5, "Ground": 1, "Flying": 0.5, "Bug": 0.5, "Rock": 2, "Ghost": 0, "Steel": 2, "Fire": 1, "Water": 1, "Electric": 1, "Grass": 1, "Ice": 2, "Psychic": 0.5, "Dragon": 1, "Dark": 2, "Fairy": 0.5}
Poison = {"Normal": 1, "Fighting": 1, "Poison": 0.5, "Ground": 0.5, "Flying": 1, "Bug": 1, "Rock": 0.5, "Ghost": 0.5, "Steel": 0, "Fire": 1, "Water": 1, "Electric": 1, "Grass": 2, "Ice": 1, "Psychic": 1, "Dragon": 1, "Dark": 1, "Fairy": 2}
Ground = {"Normal": 1, "Fighting": 1, "Poison": 2, "Ground": 1, "Flying": 0, "Bug": 0.5, "Rock": 2, "Ghost": 1, "Steel": 2, "Fire": 2, "Water": 1, "Electric": 2, "Grass": 0.5, "Ice": 1, "Psychic": 1, "Dragon": 1, "Dark": 1, "Fairy": 1}
Flying = {"Normal": 1, "Fighting": 2, "Poison": 1, "Ground": 1, "Flying": 1, "Bug": 2, "Rock": 0.5, "Ghost": 1, "Steel": 0.5, "Fire": 1, "Water": 1, "Electric": 0.5, "Grass": 2, "Ice": 1, "Psychic": 1, "Dragon": 1, "Dark": 1, "Fairy": 1}
Bug = {"Normal": 1, "Fighting": 0.5, "Poison": 0.5, "Ground": 1, "Flying": 0.5, "Bug": 1, "Rock": 1, "Ghost": 0.5, "Steel": 0.5, "Fire": 0.5, "Water": 1, "Electric": 1, "Grass": 2, "Ice": 1, "Psychic": 2, "Dragon": 1, "Dark": 2, "Fairy": 0.5}
Rock = {"Normal": 1, "Fighting": 0.5, "Poison": 1, "Ground": 0.5, "Flying": 2, "Bug": 2, "Rock": 1, "Ghost": 1, "Steel": 0.5, "Fire": 2, "Water": 1, "Electric": 1, "Grass": 1, "Ice": 2, "Psychic": 1, "Dragon": 1, "Dark": 1, "Fairy": 1}
Ghost = {"Normal": 0, "Fighting": 1, "Poison": 1, "Ground": 1, "Flying": 1, "Bug": 1, "Rock": 1, "Ghost": 2, "Steel": 1, "Fire": 1, "Water": 1, "Electric": 1, "Grass": 1, "Ice": 1, "Psychic": 2, "Dragon": 1, "Dark": 0.5, "Fairy": 1}
Steel = {"Normal": 1, "Fighting": 1, "Poison": 1, "Ground": 1, "Flying": 1, "Bug": 1, "Rock": 2, "Ghost": 1, "Steel": 0.5, "Fire": 0.5, "Water": 0.5, "Electric": 0.5, "Grass": 1, "Ice": 2, "Psychic": 1, "Dragon": 1, "Dark": 1, "Fairy": 0.5}
Fire = {"Normal": 1, "Fighting": 1, "Poison": 1, "Ground": 1, "Flying": 1, "Bug": 2, "Rock": 0.5, "Ghost": 1, "Steel": 2, "Fire": 0.5, "Water": 0.5, "Electric": 1, "Grass": 2, "Ice": 2, "Psychic": 1, "Dragon": 0.5, "Dark": 1, "Fairy": 1}
Water = {"Normal": 1, "Fighting": 1, "Poison": 1, "Ground": 2, "Flying": 1, "Bug": 1, "Rock": 2, "Ghost": 1, "Steel": 1, "Fire": 2, "Water": 0.5, "Electric": 1, "Grass": 0.5, "Ice": 1, "Psychic": 1, "Dragon": 0.5, "Dark": 1, "Fairy": 1}
Electric = {"Normal": 1, "Fighting": 1, "Poison": 1, "Ground": 0, "Flying": 2, "Bug": 1, "Rock": 1, "Ghost": 1, "Steel": 1, "Fire": 1, "Water": 2, "Electric": 0.5, "Grass": 0.5, "Ice": 1, "Psychic": 1, "Dragon": 0.5, "Dark": 1, "Fairy": 1}
Grass = {"Normal": 1, "Fighting": 1, "Poison": 0.5, "Ground": 2, "Flying": 0.5, "Bug": 0.5, "Rock": 2, "Ghost": 1, "Steel": 0.5, "Fire": 0.5, "Water": 2, "Electric": 1, "Grass": 0.5, "Ice": 1, "Psychic": 1, "Dragon": 0.5, "Dark": 1, "Fairy": 1}
Ice = {"Normal": 1, "Fighting": 1, "Poison": 1, "Ground": 2, "Flying": 2, "Bug": 1, "Rock": 1, "Ghost": 1, "Steel": 0.5, "Fire": 0.5, "Water": 0.5, "Electric": 1, "Grass": 2, "Ice": 0.5, "Psychic": 1, "Dragon": 2, "Dark": 1, "Fairy": 1}
Psychic = {"Normal": 1, "Fighting": 1, "Poison": 2, "Ground": 2, "Flying": 1, "Bug": 1, "Rock": 1, "Ghost": 1, "Steel": 0.5, "Fire": 1, "Water": 1, "Electric": 1, "Grass": 1, "Ice": 1, "Psychic": 0.5, "Dragon": 1, "Dark": 0, "Fairy": 1}
Dragon = {"Normal": 1, "Fighting": 1, "Poison": 1, "Ground": 1, "Flying": 1, "Bug": 1, "Rock": 1, "Ghost": 1, "Steel": 0.5, "Fire": 1, "Water": 1, "Electric": 1, "Grass": 1, "Ice": 1, "Psychic": 1, "Dragon": 2, "Dark": 1, "Fairy": 0}
Dark = {"Normal": 1, "Fighting": 0.5, "Poison": 1, "Ground": 1, "Flying": 1, "Bug": 1, "Rock": 1, "Ghost": 2, "Steel": 1, "Fire": 1, "Water": 1, "Electric": 1, "Grass": 1, "Ice": 1, "Psychic": 2, "Dragon": 1, "Dark": 0.5, "Fairy": 0.5}
Fairy = {"Normal": 1, "Fighting": 2, "Poison": 0.5, "Ground": 1, "Flying": 1, "Bug": 1, "Rock": 1, "Ghost": 1, "Steel": 0.5, "Fire": 0.5, "Water": 1, "Electric": 1, "Grass": 1, "Ice": 1, "Psychic": 1, "Dragon": 2, "Dark": 2, "Fairy": 1}

# Create a table where each dict entry calls on the variable dicts above
type_relation = {"Normal": Normal, "Fighting": Fighting, "Poison": Poison, "Ground": Ground, "Flying": Flying, "Bug": Bug, "Rock": Rock, "Ghost": Ghost, "Steel": Steel, "Fire": Fire, "Water": Water, "Electric": Electric, "Grass": Grass, "Ice": Ice, "Psychic": Psychic, "Dragon": Dragon, "Dark": Dark, "Fairy": Fairy}

# Create a new dataframe using the above table
df_type_relation = pd.DataFrame(type_relation)
print('Row is Defense, Column is Offense')

# Check
df_type_relation

Row is Defense, Column is Offense


Normal  Fighting  Poison  Ground  Flying  Bug  Rock  Ghost  Steel  \
Bug          1.0       0.5     1.0     0.5     2.0  1.0   2.0    1.0    1.0   
Dark         1.0       2.0     1.0     1.0     1.0  2.0   1.0    0.5    1.0   
Dragon       1.0       1.0     1.0     1.0     1.0  1.0   1.0    1.0    1.0   
Electric     1.0       1.0     1.0     2.0     0.5  1.0   1.0    1.0    0.5   
Fairy        1.0       0.5     2.0     1.0     1.0  0.5   1.0    1.0    0.5   
Fighting     1.0       1.0     1.0     1.0     2.0  0.5   0.5    1.0    1.0   
Fire         1.0       1.0     1.0     2.0     1.0  0.5   2.0    1.0    0.5   
Flying       1.0       0.5     1.0     0.0     1.0  0.5   2.0    1.0    1.0   
Ghost        0.0       0.0     0.5     1.0     1.0  0.5   1.0    2.0    1.0   
Grass        1.0       1.0     2.0     0.5     2.0  2.0   1.0    1.0    1.0   
Ground       1.0       1.0     0.5     1.0     1.0  1.0   0.5    1.0    1.0   
Ice          1.0       2.0     1.0     1.0     1.0  1.0   2.0    1.0    2.0   
Normal       1.0       2.0     1.0     1.0     1.0  1.0   1.0    0.0    1.0   
Poison       1.0       0.5     0.5     2.0     1.0  0.5   1.0    1.0    1.0   
Psychic      1.0       0.5     1.0     1.0     1.0  2.0   1.0    2.0    1.0   
Rock         0.5       2.0     0.5     2.0     0.5  1.0   1.0    1.0    2.0   
Steel        0.5       2.0     0.0     2.0     0.5  0.5   0.5    1.0    0.5   
Water        1.0       1.0     1.0     1.0     1.0  1.0   1.0    1.0    0.5   

          Fire  Water  Electric  Grass  Ice  Psychic  Dragon  Dark  Fairy  
Bug        2.0    1.0       1.0    0.5  1.0      1.0     1.0   1.0    1.0  
Dark       1.0    1.0       1.0    1.0  1.0      0.0     1.0   0.5    2.0  
Dragon     0.5    0.5       0.5    0.5  2.0      1.0     2.0   1.0    2.0  
Electric   1.0    1.0       0.5    1.0  1.0      1.0     1.0   1.0    1.0  
Fairy      1.0    1.0       1.0    1.0  1.0      1.0     0.0   0.5    1.0  
Fighting   1.0    1.0       1.0    1.0  1.0      1.0     1.0   0.5    2.0  
Fire       0.5    2.0       1.0    0.5  0.5      1.0     1.0   1.0    0.5  
Flying     1.0    1.0       2.0    0.5  2.0      1.0     1.0   1.0    1.0  
Ghost      1.0    1.0       1.0    1.0  1.0      1.0     1.0   2.0    1.0  
Grass      2.0    0.5       0.5    0.5  2.0      1.0     1.0   1.0    1.0  
Ground     1.0    2.0       0.0    2.0  2.0      2.0     1.0   1.0    1.0  
Ice        2.0    1.0       1.0    1.0  0.5      1.0     1.0   1.0    1.0  
Normal     1.0    1.0       1.0    1.0  1.0      1.0     1.0   1.0    1.0  
Poison     1.0    1.0       1.0    0.5  1.0      2.0     1.0   1.0    0.5  
Psychic    1.0    1.0       1.0    1.0  1.0      0.5     1.0   2.0    1.0  
Rock       0.5    2.0       1.0    2.0  1.0      1.0     1.0   1.0    1.0  
Steel      2.0    1.0       1.0    0.5  0.5      0.5     0.5   1.0    0.5  
Water      0.5    0.5       2.0    2.0  0.5      1.0     1.0   1.0    1.0

In order for this to be a factor in actual fights, we will make a `Type_Advantage` column with the multiplier as a numeric factor. For the sake of ensuring certain Pokemon are not rendered completely useless when they have a multplier of 0 (as is the case with certain types), we will change all 0's to 0.1.

In [14]:
# Create a column 'Type', which is a concatenation of Type 1 and Type 2
pokemon["Type"] = pokemon.apply(lambda x: x['Type 1'] + "/" + x['Type 2'], axis=1)

# Create a variable with zipped # and Type in a dict
type_dict = dict(zip(pokemon['#'], pokemon['Type']))

# Create a new table using the 'combat' table columns, replacing with above dict
combat_type = combat[['First_pokemon', 'Second_pokemon', 'Winner']].replace(type_dict)

# Add a 'Loser' column, identifying the loser of the fight
combat_type["Loser"] = combat_type.apply(lambda x: x["First_pokemon"] if x["First_pokemon"] !=  x["Winner"] else x["Second_pokemon"], axis = 1)

# Replace all 0's in the type relation dataframe above with 0.1
zero_dict = {0: 0.1}
df_type_relation = df_type_relation[:].replace(zero_dict)

# Check
combat_type.head()

First_pokemon Second_pokemon         Winner           Loser
0     Rock/Ground     Grass/Dark     Grass/Dark     Rock/Ground
1  Grass/Fighting  Rock/Fighting  Rock/Fighting  Grass/Fighting
2    Fairy/Flying   Psychic/None   Psychic/None    Fairy/Flying
3       Fire/None    Dragon/None    Dragon/None       Fire/None
4      Rock/Water       Bug/Rock     Rock/Water        Bug/Rock

#### Creating the Type Multiplier Function
<a id="multi"></a>
___

Let's create a function that actually includes the Type Multiplier in the data as a column.

In [15]:
# Define a function
def calcRelation(combat_type):
    # Label r0 as the base multiplier (unchanged)
    r0 = 1
    # Split the string of types above to isolate the first and second types for
    # both the winner and loser
    win_type1 = combat_type["Winner"].split("/")[0]
    win_type2 = combat_type["Winner"].split("/")[1]
    lose_type1 = combat_type["Loser"].split("/")[0]
    lose_type2 = combat_type["Loser"].split("/")[1]
    # Cycle through all combinations of types, and label each r(#) as
    # the multiplier for the type combination. The final r will be the 
    # culminating multiplier of each fight's type combinations.
    if win_type2 != "None" and lose_type2 != "None":
        r1 = df_type_relation[win_type1][lose_type1]
        r2 = df_type_relation[win_type1][lose_type2]
        r3 = df_type_relation[win_type2][lose_type1]
        r4 = df_type_relation[win_type2][lose_type2]
        r = r0 * r1 * r2 * r3 * r4
    elif win_type2 != "None" and lose_type2 == "None":
        r1 = df_type_relation[win_type1][lose_type1]
        r3 = df_type_relation[win_type2][lose_type1]
        r = r0 * r1 * r3
    elif win_type2 == "None" and lose_type2 != "None":
        r1 = df_type_relation[win_type1][lose_type1]
        r2 = df_type_relation[win_type1][lose_type2]
        r = r0 * r1 * r2
    elif win_type2 == "None" and lose_type2 == "None":
        r1 = df_type_relation[win_type1][lose_type1]
        r = r0 * r1
    return r

# Create a new column 'Relation', which holds the 'r' multiplier for each fight
# (assuming 'First_pokemon' is the attacker)
combat_type['Relation'] = combat_type.apply(lambda x: calcRelation(x), axis = 1)

# Check
print (combat_type["Relation"].describe())
combat_type.head()

count    50000.000000
mean         1.140050
std          0.875451
min          0.005000
25%          0.500000
50%          1.000000
75%          1.000000
max         16.000000
Name: Relation, dtype: float64


First_pokemon Second_pokemon         Winner           Loser  Relation
0     Rock/Ground     Grass/Dark     Grass/Dark     Rock/Ground       4.0
1  Grass/Fighting  Rock/Fighting  Rock/Fighting  Grass/Fighting       0.5
2    Fairy/Flying   Psychic/None   Psychic/None    Fairy/Flying       1.0
3       Fire/None    Dragon/None    Dragon/None       Fire/None       1.0
4      Rock/Water       Bug/Rock     Rock/Water        Bug/Rock       4.0

We keep the types instead of the names in these tables solely for back-checking purposes; the type advantages do check out with the real game.

## Creating the Final DataFrame
<a id="finalframe"></a>
___

The last dataframe that we will be using for model evaluation has to be a compilation of all the statistics we have above for each individual pokemon, per row. This can be done by zipping all of the relevant stats to each pokemon #.

In [16]:
# Make a copy of the combat dataframe for final use.
combat_final = combat.copy()

# Make dicts with the zipped information relevant to the stats of a pokemon.
type_dict = dict(zip(pokemon['#'], pokemon['Type']))
hp_dict = dict(zip(pokemon['#'], pokemon['HP']))
atk_dict = dict(zip(pokemon['#'], pokemon['Attack']))
def_dict = dict(zip(pokemon['#'], pokemon['Defense']))
spatk_dict = dict(zip(pokemon['#'], pokemon['Sp. Atk']))
spdef_dict = dict(zip(pokemon['#'], pokemon['Sp. Def']))
speed_dict = dict(zip(pokemon['#'], pokemon['Speed']))

# Unzip the information into the newly-made dataframe for the first pokemon.
combat_final['First_pokemon_type'] = combat_final['First_pokemon'].replace(type_dict)
combat_final['First_pokemon_hp'] = combat_final['First_pokemon'].replace(hp_dict)
combat_final['First_pokemon_atk'] = combat_final['First_pokemon'].replace(atk_dict)
combat_final['First_pokemon_def'] = combat_final['First_pokemon'].replace(def_dict)
combat_final['First_pokemon_spatk'] = combat_final['First_pokemon'].replace(spatk_dict)
combat_final['First_pokemon_spdef'] = combat_final['First_pokemon'].replace(spdef_dict)
combat_final['First_pokemon_speed'] = combat_final['First_pokemon'].replace(speed_dict)

# Unzip the information into the newly-made dataframe for the second pokemon.
combat_final['Second_pokemon_type'] = combat_final['Second_pokemon'].replace(type_dict)
combat_final['Second_pokemon_hp'] = combat_final['Second_pokemon'].replace(hp_dict)
combat_final['Second_pokemon_atk'] = combat_final['Second_pokemon'].replace(atk_dict)
combat_final['Second_pokemon_def'] = combat_final['Second_pokemon'].replace(def_dict)
combat_final['Second_pokemon_spatk'] = combat_final['Second_pokemon'].replace(spatk_dict)
combat_final['Second_pokemon_spdef'] = combat_final['Second_pokemon'].replace(spdef_dict)
combat_final['Second_pokemon_speed'] = combat_final['Second_pokemon'].replace(speed_dict)

# Define a type advantage relationship similar to how we did above. This will be added into the final dataframe.
def calcRelation(combat_final):
    # Label r0 as the base multiplier (unchanged)
    r0 = 1
    # Split the string of types above to isolate the first and second types for
    # both the first and second pokemon.
    first_pokemon_type1 = combat_final["First_pokemon_type"].split("/")[0]
    first_pokemon_type2 = combat_final["First_pokemon_type"].split("/")[1]
    second_pokemon_type1 = combat_final["Second_pokemon_type"].split("/")[0]
    second_pokemon_type2 = combat_final["Second_pokemon_type"].split("/")[1]
    # Cycle through all combinations of types, and label each r(#) as
    # the multiplier for the type combination. The final r will be the 
    # culminating multiplier of each fight's type combinations.
    if first_pokemon_type2 != "None" and second_pokemon_type2 != "None":
        r1 = df_type_relation[first_pokemon_type1][second_pokemon_type1]
        r2 = df_type_relation[first_pokemon_type1][second_pokemon_type2]
        r3 = df_type_relation[first_pokemon_type2][second_pokemon_type1]
        r4 = df_type_relation[first_pokemon_type2][second_pokemon_type2]
        r = r0 * r1 * r2 * r3 * r4
    elif first_pokemon_type2 != "None" and second_pokemon_type2 == "None":
        r1 = df_type_relation[first_pokemon_type1][second_pokemon_type1]
        r3 = df_type_relation[first_pokemon_type2][second_pokemon_type1]
        r = r0 * r1 * r3
    elif first_pokemon_type2 == "None" and second_pokemon_type2 != "None":
        r1 = df_type_relation[first_pokemon_type1][second_pokemon_type1]
        r2 = df_type_relation[first_pokemon_type1][second_pokemon_type2]
        r = r0 * r1 * r2
    elif first_pokemon_type2 == "None" and second_pokemon_type2 == "None":
        r1 = df_type_relation[first_pokemon_type1][second_pokemon_type1]
        r = r0 * r1
    return r

# Create a new column 'Relation', which holds the 'r' multiplier for each fight
# (assuming 'First_pokemon' is the attacker)
combat_final['Type_Multiplier'] = combat_final.apply(lambda x: calcRelation(x), axis = 1)

# Create a column identifying the winner. 1 if the First_pokemon won, 0 otherwise.
combat_final['First_pokemon_wins'] = combat_final.apply(lambda x: 1 if x['First_pokemon'] == x['Winner'] else 0, axis=1)

combat_final.head()

First_pokemon  Second_pokemon  Winner First_pokemon_type  First_pokemon_hp  \
0            266             298     298        Rock/Ground                50   
1            702             701     701     Grass/Fighting                91   
2            191             668     668       Fairy/Flying                55   
3            237             683     683          Fire/None                40   
4            151             231     151         Rock/Water                70   

   First_pokemon_atk  First_pokemon_def  First_pokemon_spatk  \
0                 64                 50                   45   
1                 90                 72                   90   
2                 40                 85                   80   
3                 40                 40                   70   
4                 60                125                  115   

   First_pokemon_spdef  First_pokemon_speed Second_pokemon_type  \
0                   50                   41          Grass/Dark   
1                  129                  108       Rock/Fighting   
2                  105                   40        Psychic/None   
3                   40                   20         Dragon/None   
4                   70                   55            Bug/Rock   

   Second_pokemon_hp  Second_pokemon_atk  Second_pokemon_def  \
0                 70                  70                  40   
1                 91                 129                  90   
2                 75                  75                  75   
3                 77                 120                  90   
4                 20                  10                 230   

   Second_pokemon_spatk  Second_pokemon_spdef  Second_pokemon_speed  \
0                    60                    40                    60   
1                    72                    90                   108   
2                   125                    95                    40   
3                    60                    90                    48   
4                    10                   230                     5   

   Type_Multiplier  First_pokemon_wins  
0              0.5                   0  
1              4.0                   0  
2              1.0                   0  
3              0.5                   0  
4              4.0                   1

Looking through, some of these columns are now irrelevant. Let's remove these now.

In [17]:
# Drop all columns that have irrelevant/double-counted information
combat_final2 = combat_final.drop(['First_pokemon', 'Second_pokemon', 'Winner', 'First_pokemon_type', 'Second_pokemon_type'], axis=1)

# Check
combat_final2.head()

First_pokemon_hp  First_pokemon_atk  First_pokemon_def  \
0                50                 64                 50   
1                91                 90                 72   
2                55                 40                 85   
3                40                 40                 40   
4                70                 60                125   

   First_pokemon_spatk  First_pokemon_spdef  First_pokemon_speed  \
0                   45                   50                   41   
1                   90                  129                  108   
2                   80                  105                   40   
3                   70                   40                   20   
4                  115                   70                   55   

   Second_pokemon_hp  Second_pokemon_atk  Second_pokemon_def  \
0                 70                  70                  40   
1                 91                 129                  90   
2                 75                  75                  75   
3                 77                 120                  90   
4                 20                  10                 230   

   Second_pokemon_spatk  Second_pokemon_spdef  Second_pokemon_speed  \
0                    60                    40                    60   
1                    72                    90                   108   
2                   125                    95                    40   
3                    60                    90                    48   
4                    10                   230                     5   

   Type_Multiplier  First_pokemon_wins  
0              0.5                   0  
1              4.0                   0  
2              1.0                   0  
3              0.5                   0  
4              4.0                   1

And now we have the final dataframe. Only a few more steps left.

## Creating the Model / Prediction System
<a id="predict"></a>
___

Now that we have finalized a dataframe to use, we can now optimize the hyperparameters for a predictive model. However, we must first go through a train-test split to verify the accuracy of our results.

In [18]:
from sklearn.model_selection import train_test_split

# Indicate the variables. Since we unfortunately have categorical indices as numeric values, we cannot make simple slices.
X = combat_final2.drop('First_pokemon_wins', axis=1)
y = combat_final2['First_pokemon_wins']

# Train-test split (keeping random state for repeatability)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=0)

Next, we will start using multiple different models to obtain the highest test accuracy. Something to keep in mind is that the variables selected are arbitrarily chosen. The ideal method is to optimize hyperparameters using a GridSearch method, but this is extremely taxing when combing through multiple values for an array of variables. For future work, I would very much wish to use a GridSearch over a large period of time.

In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Instantiate the models
pokelog = LogisticRegression(solver='lbfgs')
pokeknn = KNeighborsClassifier(n_neighbors = 10)
poketree = DecisionTreeClassifier(max_depth = 10)
pokerndm = RandomForestClassifier(n_estimators = 100)

# Fit
pokelog.fit(X_train, y_train)
pokeknn.fit(X_train, y_train)
poketree.fit(X_train, y_train)
pokerndm.fit(X_train, y_train)

# Score and print
print(f'Logistic Regression test score = {pokelog.score(X_test, y_test) * 100}')
print(f'K Nearest Neighbors test score = {pokeknn.score(X_test, y_test) * 100}')
print(f'Decision Tree test score = {poketree.score(X_test, y_test) * 100}')
print(f'Random Forest test score = {pokerndm.score(X_test, y_test) * 100}')

Logistic Regression test score = 88.53999999999999
K Nearest Neighbors test score = 88.2
Decision Tree test score = 94.72
Random Forest test score = 95.38


We see that the Random Forest test gives the highest test score with 95.38%. Given this, we can use the Random Forest fit to predict the winner of the remaining test fights. In order to do this, we have to set up the data similar to how we set up the training set. But first, let's take a look at the key factors that play a part in the model.

In [20]:
# Create a new empty dataframe.
feature_values = pd.DataFrame()
# Fill a column named 'feature_name' with the features from the X values.
feature_values['feature_name'] = X.columns.tolist()
# Extract the feature coefficients into a new column.
feature_values['feature_importance'] = pokerndm.feature_importances_
# Finally, sort the feature importances in descending order.
feature_values.sort_values('feature_importance', ascending=False)

feature_name  feature_importance
5    First_pokemon_speed            0.309929
11  Second_pokemon_speed            0.300613
1      First_pokemon_atk            0.053002
7     Second_pokemon_atk            0.047428
3    First_pokemon_spatk            0.041231
9   Second_pokemon_spatk            0.037746
12       Type_Multiplier            0.037288
0       First_pokemon_hp            0.031116
6      Second_pokemon_hp            0.029900
4    First_pokemon_spdef            0.028574
10  Second_pokemon_spdef            0.028255
2      First_pokemon_def            0.027970
8     Second_pokemon_def            0.026946

As seen above, Speed seems to be one of the strongest variables to consider. This actually makes sense from a competitive standpoint; more often than not, a Pokemon with the first move is more likely to win. This is particularly true in the higher levels of competitive play.

Now, how about predicting the winners?

In [21]:
# Create the dataframes for X_test and y_predict
y_predict = pd.DataFrame(pokerndm.predict(X_test), columns=['Winner'])
X_predict_data = pd.DataFrame(X_test)

# Reset indices so we can concatenate
y_predict.reset_index(drop=True, inplace=True)
X_predict_data.reset_index(drop=True, inplace=True)

# Concatenate
winner_predict = pd.concat([X_predict_data, y_predict], axis=1)
print(winner_predict.shape)
winner_predict.head()

(10000, 14)


First_pokemon_hp  First_pokemon_atk  First_pokemon_def  \
0                60                 85                 49   
1                62                 48                 54   
2                60                 90                 70   
3                95                 23                 48   
4                45                 60                 32   

   First_pokemon_spatk  First_pokemon_spdef  First_pokemon_speed  \
0                   60                   49                   60   
1                   63                   60                   68   
2                   60                  120                   40   
3                   23                   48                   23   
4                   50                   32                   76   

   Second_pokemon_hp  Second_pokemon_atk  Second_pokemon_def  \
0                 40                  45                  35   
1                 65                  83                  57   
2                 60                  69                  95   
3                 55                 112                  45   
4                 20                  15                  20   

   Second_pokemon_spatk  Second_pokemon_spdef  Second_pokemon_speed  \
0                    65                    55                    70   
1                    95                    85                   105   
2                    69                    95                    36   
3                    74                    45                    70   
4                    10                    55                    80   

   Type_Multiplier  Winner  
0              0.5       0  
1              1.0       0  
2              0.5       1  
3              1.0       0  
4              2.0       0

And there we have it; we have successfully created a model and predicted the winning Pokemon in the test dataset with a 95.38% accuracy. So, what about putting all of this together into a function?

In [58]:
def pokematch(poke1, poke2):
    # Take the input pokemon numbers, and create an X_value dataframe to use for predictions
    combat_func = pd.DataFrame(columns=['First_pokemon', 'Second_pokemon'])
    combat_func = combat_func.append({'First_pokemon': poke1, 'Second_pokemon': poke2}, ignore_index=True)
    
    # Make dicts with the zipped information relevant to the stats of a pokemon.
    type_dict_func = dict(zip(pokemon['#'], pokemon['Type']))
    hp_dict_func = dict(zip(pokemon['#'], pokemon['HP']))
    atk_dict_func = dict(zip(pokemon['#'], pokemon['Attack']))
    def_dict_func = dict(zip(pokemon['#'], pokemon['Defense']))
    spatk_dict_func = dict(zip(pokemon['#'], pokemon['Sp. Atk']))
    spdef_dict_func = dict(zip(pokemon['#'], pokemon['Sp. Def']))
    speed_dict_func = dict(zip(pokemon['#'], pokemon['Speed']))

    # Unzip the information into the newly-made dataframe for the first pokemon.
    combat_func['First_pokemon_type'] = combat_func['First_pokemon'].replace(type_dict_func)
    combat_func['First_pokemon_hp'] = combat_func['First_pokemon'].replace(hp_dict_func)
    combat_func['First_pokemon_atk'] = combat_func['First_pokemon'].replace(atk_dict_func)
    combat_func['First_pokemon_def'] = combat_func['First_pokemon'].replace(def_dict_func)
    combat_func['First_pokemon_spatk'] = combat_func['First_pokemon'].replace(spatk_dict_func)
    combat_func['First_pokemon_spdef'] = combat_func['First_pokemon'].replace(spdef_dict_func)
    combat_func['First_pokemon_speed'] = combat_func['First_pokemon'].replace(speed_dict_func)

    # Unzip the information into the newly-made dataframe for the second pokemon.
    combat_func['Second_pokemon_type'] = combat_func['Second_pokemon'].replace(type_dict_func)
    combat_func['Second_pokemon_hp'] = combat_func['Second_pokemon'].replace(hp_dict_func)
    combat_func['Second_pokemon_atk'] = combat_func['Second_pokemon'].replace(atk_dict_func)
    combat_func['Second_pokemon_def'] = combat_func['Second_pokemon'].replace(def_dict_func)
    combat_func['Second_pokemon_spatk'] = combat_func['Second_pokemon'].replace(spatk_dict_func)
    combat_func['Second_pokemon_spdef'] = combat_func['Second_pokemon'].replace(spdef_dict_func)
    combat_func['Second_pokemon_speed'] = combat_func['Second_pokemon'].replace(speed_dict_func)

    # Define a type advantage relationship similar to how we did above. This will be added into the final dataframe.
    def calcRelation(combat_func):
        # Label r0 as the base multiplier (unchanged)
        r0 = 1
        # Split the string of types above to isolate the first and second types for
        # both the first and second pokemon.
        first_pokemon_type1_func = combat_func["First_pokemon_type"].split("/")[0]
        first_pokemon_type2_func = combat_func["First_pokemon_type"].split("/")[1]
        second_pokemon_type1_func = combat_func["Second_pokemon_type"].split("/")[0]
        second_pokemon_type2_func = combat_func["Second_pokemon_type"].split("/")[1]
        # Cycle through all combinations of types, and label each r(#) as
        # the multiplier for the type combination. The final r will be the 
        # culminating multiplier of each fight's type combinations.
        if first_pokemon_type2_func != "None" and second_pokemon_type2_func != "None":
            r1 = df_type_relation[first_pokemon_type1_func][second_pokemon_type1_func]
            r2 = df_type_relation[first_pokemon_type1_func][second_pokemon_type2_func]
            r3 = df_type_relation[first_pokemon_type2_func][second_pokemon_type1_func]
            r4 = df_type_relation[first_pokemon_type2_func][second_pokemon_type2_func]
            r = r0 * r1 * r2 * r3 * r4
        elif first_pokemon_type2_func != "None" and second_pokemon_type2_func == "None":
            r1 = df_type_relation[first_pokemon_type1_func][second_pokemon_type1_func]
            r3 = df_type_relation[first_pokemon_type2_func][second_pokemon_type1_func]
            r = r0 * r1 * r3
        elif first_pokemon_type2_func == "None" and second_pokemon_type2_func != "None":
            r1 = df_type_relation[first_pokemon_type1_func][second_pokemon_type1_func]
            r2 = df_type_relation[first_pokemon_type1_func][second_pokemon_type2_func]
            r = r0 * r1 * r2
        elif first_pokemon_type2_func == "None" and second_pokemon_type2_func == "None":
            r1 = df_type_relation[first_pokemon_type1_func][second_pokemon_type1_func]
            r = r0 * r1
        return r

    # Create a new column 'Relation', which holds the 'r' multiplier for each fight
    # (assuming 'First_pokemon' is the attacker)
    combat_func['Type_Multiplier'] = combat_func.apply(lambda x: calcRelation(x), axis = 1)
    
    # Drop all columns that have irrelevant/double-counted information
    combat_func = combat_func.drop(['First_pokemon', 'Second_pokemon', 'First_pokemon_type', 'Second_pokemon_type'], axis=1)
    
    # Create the dataframes for X_test and y_predict
    y_predict = pd.DataFrame(pokerndm.predict(combat_func), columns=['Winner'])
    X_predict_data = pd.DataFrame(combat_func)
    
    # Reset indices so we can concatenate
    y_predict.reset_index(drop=True, inplace=True)
    X_predict_data.reset_index(drop=True, inplace=True)

    # Concatenate
    winner_predict = pd.concat([X_predict_data, y_predict], axis=1)
    
    if winner_predict['Winner'][0] == 1:
        print('The winner is' + '\033[1m' + ' Pokemon 1!')
    else:
        print('The winner is' + '\033[1m' + ' Pokemon 2!')
        
    return winner_predict

# TRY IT OUT!
<a id="testit"></a>
___

Simply enter any 2 numbers with the `pokematch` function, as indicated below. You can enter any number between 1-800.

Remember, if the `Winning` column indicates 1, then the First Pokemon is predicted to win the fight. Otherwise, 0.

In [68]:
# Format: pokematch(random_number_1, random_number_2)
pokematch(158, 645)

The winner is Pokemon 1!


First_pokemon_hp  First_pokemon_atk  First_pokemon_def  \
0                90                 90                 85   

   First_pokemon_spatk  First_pokemon_spdef  First_pokemon_speed  \
0                  125                   90                  100   

   Second_pokemon_hp  Second_pokemon_atk  Second_pokemon_def  \
0                 51                  65                  65   

   Second_pokemon_spatk  Second_pokemon_spdef  Second_pokemon_speed  \
0                    80                    75                    59   

   Type_Multiplier  Winner  
0              1.0       1

If you want to check what Pokemon you have pit against each other, you can search them below:

In [71]:
# Change the number to the Pokemon ID number you wish to search 
# (i.e. if you want to find ID 25: pokemon[pokemon['#'] == 25])
pokemon[pokemon['#'] == 158]

#    Name    Type 1  Type 2  HP  Attack  Defense  Sp. Atk  Sp. Def  \
157  158  Zapdos  Electric  Flying  90      90       85      125       90   

     Speed  Generation  Legendary             Type  
157    100           1       True  Electric/Flying

## Future Steps
<a id="future"></a>
___

While this particular model predicts victory of individual bouts between Pokemon, classic Pokemon battles take place in a 6 versus 6 King-of-the-Hill style format. Furthermore, there are many other considerations to include, such as the ability to freely switch Pokemon in the middle of the battle, restrictions of Pokemon based on tier rankings, and much more. While these other factors extend past the scope of this project, we can expect this current model to be applicable in future endeavors. Some examples include:

1. Allowing multiple Pokemon in a single team.
2. As per original format, having Pokemon fight in a King-of-the-Hill style.
3. Including Pokemon movesets (4 different Pokemon moves) that include Same-Type Advantage Bonus.
4. Allowing held items.
5. And much more...

The above ideas are extremely time consuming, and there are some factors that hold no codeable logic (i.e. intentionally switching into a more disadvantageous Pokemon). With time, I am certain we can piece together models and AI that could successfully incorporate these methods.

# Thanks for reading!